In [1]:
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import os
import sys

In [2]:
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

from src.util import mmss_to_ms, get_chapter_mapping_df, get_chapters_df, get_transcription_df

In [3]:
# Paths
load_dotenv()

TSV = Path(os.getenv("VIDEO_DATA_TSV")).resolve()
CHAPTERS_DIR = Path(os.getenv("CHAPTERS_DIR")).resolve()
TRANSCRIPTIONS_DIR = Path(os.getenv("TRANSCRIPTIONS_DIR")).resolve()
CHAPTERS_TIMESTAMPED_DIR = Path(os.getenv("CHAPTERS_TIMESTAMPED_DIR")).resolve()
assert TSV.is_file(), "Could not find TSV."
assert CHAPTERS_DIR.is_dir(), "Could not find chapters directory."
assert TRANSCRIPTIONS_DIR.is_dir(), "Could not find transcriptions directory."
CHAPTERS_TIMESTAMPED_DIR.mkdir(parents=False, exist_ok=True)
assert CHAPTERS_TIMESTAMPED_DIR.is_dir(), "Could not find directory for timestamped chapters."

TARGET_FILE = Path(os.getenv("CHAPTERS_DATA_TSV")).resolve()

COLUMNS_TO_KEEP = [
    "title",
    "chapter",
    "start_mm:ss",
    "end_mm:ss",
    "content",
    "audio_transcription",
    "filestem",
    "episode",
    "year",
    "end",
    "start",
]

In [ ]:
def get_transcription_for_chapter(
    df: pd.DataFrame, start: float | int | None, end: float | int | None
) -> str:
    if start is None or end is None or pd.isna(start) or pd.isna(end):
        return ""
    mask = (df["start"] < end) & (df["end"] > start)
    texts = (
        df.loc[mask, "text"]
        .dropna()  # removes NaN / pd.NA
        .astype(str)  # ensures strings
        .str.strip()  # removes ""
    )

    texts = texts[texts != ""]  # drop empty strings

    return " ".join(texts)
    
movies_df = pd.read_csv(TSV, sep="\t")
# Only use data where transcription and chapters exist
movies_df = movies_df[(movies_df["has_transcription"]) & (movies_df["has_chapters"])]
n = len(movies_df)
chapters_dfs = []
for i, row in movies_df.iterrows():

    print(f"[{i+1}/{n}]{row["title"]}\n")

    # Ground truth: Chapters
    # For each movie, search chapters
    chapters_df = get_chapters_df(row["filestem"])
    chapters_df["title"] = row["title"]
    chapters_df["filestem"] = row["filestem"]
    chapters_df["episode"] = row["episode"]
    chapters_df["year"] = row["year"]
    chapters_df["chapter"] = chapters_df["chapter"].astype(str)

    # Merge timestamp mapping to chapters
    timestamp_mapping_df = get_chapter_mapping_df(row["filestem"])
    if not timestamp_mapping_df.empty:
        timestamp_mapping_df["chapter"] = timestamp_mapping_df["chapter"].astype(str)
        chapters_df = chapters_df.merge(timestamp_mapping_df, on="chapter", how="left")
        chapters_df["start"] = chapters_df["start_mm:ss"].fillna("").apply(mmss_to_ms)
        chapters_df["end"] = chapters_df["end_mm:ss"].fillna("").apply(mmss_to_ms)

    else:
        print("No timestamp mapping found.")

    # Append transcriptions
    transcriptions_df = get_transcription_df(row["filestem"])
    chapters_df["audio_transcription"] = chapters_df.apply(lambda row: get_transcription_for_chapter(transcriptions_df, row["start"], row["end"]), axis=1)

    chapters_dfs.append(chapters_df)

result = pd.concat(chapters_dfs)

[1/189]Die Deutsche Wochenschau Nr. 511 / 1940

chapter: 46000 133000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 166000 229000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 231000 274000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 308000 541000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 563000 605000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 619000 640000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 664000 731000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 759000 999000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 1010000 1036000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 1037000 1115000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 1148000 1182000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 1219000 1237000
Transcription min/max: 46800 2516610
Any overlap? True
chapter: 

In [5]:
result["audio_transcription"]

0    Der Reich ist in voller Schärfe entbrannt. Wie...
1    Bombe für Bombe trifft ihr Ziel. Der rechte Fl...
2    In Oase-Mont hat der Gegner die Tanks einer gr...
3    Im Somme-Gebiet sind heiße Kämpfe entbrannt. A...
4    Fekin ist erreicht. Eine wichtige Etappe auf d...
                           ...                        
4    Um die Abwehroperationen der schwer ringenden ...
5    Die militärische Bedeutung der deutschen seit ...
6    Am Weihnachtsabend standen die deutschen Besat...
7    Aus den machtvollen Vorstößen unserer Panzer- ...
8    In den frühen Morgenstunden des 1. Januar holt...
Name: audio_transcription, Length: 2107, dtype: str

In [6]:
result[result["start"].isna()]

,chapter,content,title,filestem,episode,year,start_mm:ss,end_mm:ss,start,end,audio_transcription
4,4,"4. Britische Truppen in Gibraltar, 1940 Wehend...",Die Deutsche Wochenschau Nr. 516 / 1940,516_1940,516,1940,NaN,NaN,NaN,NaN,
5,5,"5. Britische Truppen in Britisch Hongkong, 194...",Die Deutsche Wochenschau Nr. 516 / 1940,516_1940,516,1940,NaN,NaN,NaN,NaN,
6,6,"6. Deutsche „Zivilgefangene“, Großbritannien, ...",Die Deutsche Wochenschau Nr. 516 / 1940,516_1940,516,1940,NaN,NaN,NaN,NaN,
7,7,7. Deutsche Feuerwehr bei der Brandbekämpfung ...,Die Deutsche Wochenschau Nr. 516 / 1940,516_1940,516,1940,NaN,NaN,NaN,NaN,
8,8,"8. Aufräumarbeiten in Rouen, Frankreich, 1940 ...",Die Deutsche Wochenschau Nr. 516 / 1940,516_1940,516,1940,NaN,NaN,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...
2,3,"3. Remonte Schule in Demmin(?), Pommern, Deuts...",Die Deutsche Wochenschau Nr. 723 / 1944,723_1944,723,1944,NaN,NaN,NaN,NaN,
3,4,"4. Kunst, Prag, annektierte Tschechoslowakei, ...",Die Deutsche Wochenschau Nr. 723 / 1944,723_1944,723,1944,NaN,NaN,NaN,NaN,
5,6,"6. Finnische Front, Finnland, 1944. Kartentric...",Die Deutsche Wochenschau Nr. 723 / 1944,723_1944,723,1944,NaN,NaN,NaN,NaN,
6,7,"7. Operation Overlord, Normandie, Frankreich, ...",Die Deutsche Wochenschau Nr. 723 / 1944,723_1944,723,1944,NaN,NaN,NaN,NaN,


In [8]:
result.to_csv(TARGET_FILE, index=False, sep="\t")